In [16]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import catboost as ctb
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.linear_model import LassoCV
import lightgbm as lgbm
import numpy as np
import pandas as pd
from preprocess_block import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import glob
import re
from sklearn.svm import SVR


train = pd.read_csv("../train_with_ido.csv")
train = train.dropna(subset=["賃料"])
train = train.drop(20926)
train = train.drop(20231)
train = train.drop(20427)
train = train.drop(5775)
train = train.drop("Unnamed: 0",axis=1)

test = pd.read_csv("../test_with_ido.csv")
test = test.drop("Unnamed: 0",axis=1)

train_height = pd.read_csv("../train_height.csv",header=None)
train_height = train_height.values[0][:-1]
test_height = pd.read_csv("../test_height.csv",header=None)
test_height = test_height.values[0][:-1]

train = train.assign(field_height=train_height)
test = test.assign(field_height=test_height)

train_x = train.drop("賃料",axis=1)
train_y = train["賃料"]

rp = tree_preprocess(7777)
rpstep = rp.steps
tree_steps = [
    ("pre",Pipeline(steps=rpstep)),
    ("dummy",dummy())
]

rp = linear_preprocess()
rpstep = rp.steps
linear_steps = [
    ("pre",Pipeline(steps=rpstep)),
    ("dummy",dummy())
]

tree_pre = Pipeline(steps=tree_steps)
lin_pre = Pipeline(steps=linear_steps)

tree_pre.fit(train_x,train_y)
lin_pre.fit(train_x,train_y)

tree_x = tree_pre.predict(train_x)
lin_x = lin_pre.predict(train_x)
sc_lin_x = scale(lin_x)

tree_test = tree_pre.predict(test)

In [136]:
tree_test = tree_pre.predict(test)

from sklearn.linear_model import LassoCV

samples = 7

temp = glob.glob("./class_stack/*")
subpat = re.compile("(./class_stack/|./stacked/)")

buf = []
pred_all = []
for i in range(samples):
    txt = subpat.sub("",temp[i])
    seed = re.match(r"[0-9]+",txt)[0]
    
    hoge = np.loadtxt("./stacked/"+str(seed)+"train.csv")
    piyo = np.loadtxt("./class_stack/"+str(seed)+"_class_train.csv")
#     
    fuga = piyo[:,8:12]
    fuga = np.mean(fuga,axis=1).reshape(-1,1)
    hoge[:,24:28] = hoge[:,4:8]*fuga
    
    fuga = piyo[:,0:4]
    fuga = np.mean(fuga,axis=1).reshape(-1,1)
    hoge[:,4:8] = hoge[:,4:8]*fuga
    
    fuga = piyo[:,4:8]
    fuga = np.mean(fuga,axis=1).reshape(-1,1)
    hoge[:,8:12] = hoge[:,8:12]*fuga
    
    hoge = np.concatenate([hoge,piyo],axis=1)
    
    hoge = scale(hoge)
    hoge = np.delete(hoge,[1, 4, 5, 8, 9, 10, 11, 12, 13, 15, 16,
                           18, 19, 21, 22, 23, 24, 25, 26, 27, 28,
                           29, 30, 32, 33, 34, 35, 37, 38, 41, 42],axis=1)
    
    model = Lasso(random_state=7777,max_iter=10000,positive=True,alpha=10)
    model.fit(hoge,train_y)
    
    hoge = np.loadtxt("./stacked/"+str(seed)+"test.csv")
    piyo = np.loadtxt("./class_stack/"+str(seed)+"_class_test.csv")
#     
    fuga = piyo[:,8:12]
    fuga = np.mean(fuga,axis=1).reshape(-1,1)
    hoge[:,24:28] = hoge[:,4:8]*fuga
    
    fuga = piyo[:,0:4]
    fuga = np.mean(fuga,axis=1).reshape(-1,1)
    hoge[:,4:8] = hoge[:,4:8]*fuga
    
    fuga = piyo[:,4:8]
    fuga = np.mean(fuga,axis=1).reshape(-1,1)
    hoge[:,8:12] = hoge[:,8:12]*fuga
    
    hoge = np.concatenate([hoge,piyo],axis=1)
    
    hoge = scale(hoge)
    hoge = np.delete(hoge,[1, 4, 5, 8, 9, 10, 11, 12, 13, 15, 16,
                           18, 19, 21, 22, 23, 24, 25, 26, 27, 28,
                           29, 30, 32, 33, 34, 35, 37, 38, 41, 42],axis=1)
    pred = model.predict(hoge)
#     print(seed)
#     scores = cross_val_score(,hoge,train_y,scoring="neg_mean_squared_error",cv=4)
#     scores = np.sqrt(-np.array(scores))
#     print(scores)
#     print(np.mean(scores))

    
    pred_all.append(pred)
pred_all = np.array(pred_all)
pred = np.mean(pred_all,axis=0)
pred = pd.DataFrame(pred)
pred.columns=["pred"]
pred.index = test.index
pred = pd.concat([test["id"],pred],axis=1)
pred.to_csv("final_sub.csv",header=False,index=False)


In [ ]:
1997
[16337.99983831 14889.99800445 15024.39239171 12972.99246695]
14806.34567535379
8888
[16115.1132383  14799.39001815 15065.4364283  12987.15187551]
14741.772890064385
5555
[16339.98017893 14634.11268197 14973.09429393 12763.93826199]
14677.781354207746
1719
[16142.22331116 14901.80751391 15207.91266322 12934.28055816]
14796.556011613393
7777
[16018.21382125 14696.55825358 15219.97512567 13055.14735897]
14747.473639867092
7777
[16018.21382125 14696.55825358 15219.97512567 13055.14735897]
14747.473639867092
317
[16226.85484568 15099.29598543 15477.92927736 13323.2468622 ]
15031.831742665394

In [129]:
print(clf.alpha_)

7.943282347242366


In [ ]:
ans = []
for i in range(len(buf)):
    if buf[i] == 0:
        ans.append(i)

In [ ]:
print(ans)

In [ ]:
[38862.1131791  31239.275001   36210.80308198 30431.74958923]
34185.985212824664
C8192 eps 0.1
[37438.86370286 29631.83440891 34315.00666053 28752.31851029]
32534.50582064814
C100000 eps 0.3
[28472.32399159 18427.70794033 23163.08479282 19680.092542  ]
22435.80231668332